In [82]:
import requests
import re
from bs4 import BeautifulSoup
import time
import json

from lxml import etree
import pandas as pd


In [2]:
def from_douban_get_imdbid(douban_id : str):
    import requests
    from bs4 import BeautifulSoup
    import time
    url = "https://movie.douban.com/subject/" + douban_id
    douban_header = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36"
    }
    cookie = {
        'cookie': 'll="118254"; bid=JjNjUx_X__A; __utmz=30149280.1663765422.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); _vwo_uuid_v2=D75064BA3572224A69942C478F080A99F|b5336ef789adc779a8ab04bd6fb58743; viewed="35681856"; gr_user_id=d107d974-a76c-4062-bf2c-12014ea12e86; dbcl2="191413562:6X2MeYrmrR4"; push_noty_num=0; push_doumail_num=0; __utmv=30149280.19141; ck=cscU; __utmc=30149280; __utmc=223695111; __utma=30149280.68016018.1663765422.1664020362.1664028125.10; __utmt=1; __utmb=30149280.2.10.1664028125; __utma=223695111.658081827.1663765422.1664020362.1664028241.9; __utmb=223695111.0.10.1664028241; __utmz=223695111.1664028241.9.4.utmcsr=douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/; _pk_ses.100001.4cf6=*; _pk_ref.100001.4cf6=["","",1664028241,"https://www.douban.com/"]; _pk_id.100001.4cf6=2e9ec54f24bd52d3.1663765421.9.1664028244.1664023821.; ap_v=0,6.0'
    }
    try:
        resp = requests.get(url=url, headers=douban_header , cookies = cookie)
    except requests.exceptions.ConnectionError as e:
        time.sleep(5)
        resp = requests.get(url=url, headers=douban_header)
    if resp.status_code != 200:
        resp = requests.get(url=url, headers=douban_header, cookie=cookie)
    douban_soup = BeautifulSoup(resp.text)
    try:
        text = douban_soup.find("div" , attrs = {"id" : "info"}).text.replace("\n" , "")
    except AttributeError as e:
        return ""
    if "IMDb" in text:
        return re.findall("IMDb: tt[0-9]*" , text)[0].replace("IMDb: " , "")
    else:
        return ""




In [3]:
ids = set()

with open("./douban/html.html" , "r", encoding="utf-8") as f:
    lis_text = f.read()

In [ ]:
lis_soup = BeautifulSoup(lis_text)
lis = lis_soup.find("ul" , attrs = {"class" : "explore-list"}).find_all("li" , recursive=False)
for li in lis:
    ids.add(re.findall("[0-9]+" ,li.find("a" , recursive=False).attrs['href'])[0])
# print(ids)
result = []
for index , i in enumerate(ids):
    print("processing movie " + str(index + 1))
    imdbid = from_douban_get_imdbid(i)
    print("   " + imdbid)
    result.append([i , imdbid])

In [19]:
result_out = pd.DataFrame(data=result , columns=['douban id' , 'imdb id'])
result_out[result_out['imdb id'] != ""].to_csv('./ing_data/all_movie_list.csv' , index=False)

In [20]:
movie_list = pd.read_csv("./ing_data/all_movie_list.csv")

In [25]:
# https://www.imdb.com/title/tt7362036/fullcredits

url = "https://www.imdb.com/title/tt7362036/fullcredits"
header = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36",
    }
try:
    resp = requests.get(url=url, headers=header)
except requests.exceptions.ConnectionError as e:
    time.sleep(10)
    resp = requests.get(url=url, headers=header)
resp.raise_for_status()

soup = BeautifulSoup(resp.text)


In [28]:
title_l = soup.find("div", attrs={'id': 'fullcredits_content'}).find_all("h4")
table_l = soup.find("div", attrs={'id': 'fullcredits_content'}).find_all('table')

In [48]:
for i , u in enumerate(title_l):
    text = u.attrs.get("id")
    if text == "cast":
        c_list = table_l[i].find_all("tr" , class_ = ["odd","even"])

In [56]:
for j,v in enumerate(c_list):
    if j >= 5:
        break
    name = v.find_all("td")[1].text.strip()
    ID = re.findall('nm\d+' , v.find_all("a")[1].attrs.get("href"))[0]
    print(ID)

nm1905770
nm2489423
nm4369372
nm3431007
nm9636805


In [58]:
# https://www.imdb.com/name/nm1905770/
url_person = "https://www.imdb.com/name/nm1905770/"
header_person = {
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36"
        }
try:
    resp_person = requests.get(url=url_person, headers=header_person)
except requests.exceptions.ConnectionError as e:
    time.sleep(10)
    resp_person = requests.get(url=url_person, headers=header_person)
resp_person.raise_for_status()

soup_person = BeautifulSoup(resp_person.text)

In [60]:
l_person = soup_person.find("div" , attrs = {"id":"filmography"}).find_all("div" , recursive= False)
for i , v in enumerate(l_person):
    if v.attrs.get("id") == "filmo-head-actor" or v.attrs.get('id') == "filmo-head-actress":
        

[<div class="head" data-category="actor" id="filmo-head-actor" onclick="toggleFilmoCategory(this);">
 <span class="hide-link" id="hide-actor">Hide <img alt="Hide" class="absmiddle" height="16" src="https://m.media-amazon.com/images/S/sash/5o6LtZOQM$gRIYv.png" width="18"/></span>
 <span class="show-link" id="show-actor">Show <img alt="Show" class="absmiddle" height="16" src="https://m.media-amazon.com/images/S/sash/OPcpIfkmvSt2$q1.png" width="18"/></span>
 <a name="actor">Actor</a> (42 credits)
 </div>,
 <div class="filmo-category-section">
 <div class="filmo-row odd" id="actor-tt16757480">
 <span class="year_column">
  2022
 </span>
 <b><a href="/title/tt16757480/?ref_=nm_flmg_act_1">Nin Hao Bei Jing</a></b>
 <br/>
 Yu Xingrong
 </div>
 <div class="filmo-row even" id="actor-tt15783462">
 <span class="year_column">
  2022
 </span>
 <b><a href="/title/tt15783462/?ref_=nm_flmg_act_2">Qi ji · Ben xiao hai</a></b>
 <br/>
 </div>
 <div class="filmo-row odd" id="actor-tt16606348">
 <span clas

In [84]:
with open("./ing_data/_person_old.json") as f:
    current_film = json.load(f)

In [99]:
re.findall(r"\d{4}" , "(I) (2017)")

['2017']

In [109]:
l = re.findall(r"\d+" , "(2022)")

In [115]:
df = pd.read_csv('./out/film.txt' , sep=",")

In [121]:
df = df.drop_duplicates()